# ESMIF

## Import dependencies

In [ ]:
import os

import pandas as pd
import torch
from Bio.SeqIO.FastaIO import SimpleFastaParser
from tqdm.auto import tqdm

from pdmodels import ESMIF
from pdmodels.globals import ROOT_DIR
from pdmodels.utils import get_dms_libary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

esmif = ESMIF(device=device)

## Load data

In [ ]:
seqs_path = os.path.join(ROOT_DIR, "assets", "multimers", "multimers.fasta")
pdb_path = os.path.join(ROOT_DIR, "assets", "multimers", "NbALFA_ALFAtag_AF3.pdb")

with open(seqs_path, "r") as f:
    seqs_list = [seq for _, seq in SimpleFastaParser(f)]

print(f"Multimer sequences list: {seqs_list}")

## Score sequences

In [ ]:
output_dict = esmif.score(pdb_path, seqs_list)
output_dict

## DMS screen

In [ ]:
seqs_wt = "SGEVQLQESGGGLVQPGGSLRLSCTASGVTISALNAMAMGWYRQAPGERRVMVAAVSERGNAMYRESVQGRFTVTRDFTNKMVSLQMDNLKPEDTAVYYCHVLEDRVDSFHDYWGQGTQVTVSS:PSRLEEELRRRLTEP"
data = []
dms_library = get_dms_libary(seqs_wt)

batch_size = 32
for i in tqdm(range(0, len(dms_library), batch_size)):
    batch = dms_library[i : i + batch_size]
    batch_mutation_list, batch_seqs_list = zip(*batch)
    batch_perplexity_list = esmif.score(pdb_path, batch_seqs_list)["perplexity"].tolist()
    data.extend(list(zip(batch_mutation_list, batch_perplexity_list)))

df = pd.DataFrame(data, columns=["mutation", "perplexity"])
df

## Sample sequences

In [ ]:
output_path = os.path.join(ROOT_DIR, "assets", "multimers", "esmif_samples.fasta")
redisigned_residues = "1 3 4 5 7 8 9 13 14 15 19 20 21 23 24 25 26 27 39 41 44 45 46 48 50 52 53 67 68 69 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 88 89 91 92 93 95 97 99 100 102 114 116 118 119 120 121 123 124"

for batch_num in tqdm(range(4)):
    esmif.sample(
        pdb_path,
        output_path,
        target_chain_id="A",
        batch_size=32,
        redesigned_residues=redisigned_residues,
        omit_aa="C",
        temperature=1.0,
        padding_length=10,
        index_offset=batch_num * 32,
    )